<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20neuronal%20con%20text%20embeddings%20y%20features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
import numpy as np
import tensorflow as tf
import pandas as pd
from google.colab import files

!pip install -q tensorflow-hub
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [150]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [151]:
# LIMPIAMOS Y CHUSMEAMOS
train_df = train_df.drop(['keyword', 'location'], axis = 1)
train_df.shape

(7613, 3)

In [152]:
target_df = train_df.pop('target')

In [153]:
# SE DIVIDE UNA PARTE DEL SET PARA PODER EVALUAR LOS RESULTADOS

train_x, validation_x, train_y, validation_y = train_test_split(train_df, target_df, test_size=0.3, random_state=0)

In [154]:
# HACEMOS UN EMBEDDING PROVISTO POR TensorFlow Hub, por fuera del modelo. Al modelo le llega un array,
# de esa forma puedo agregarle features al array

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding_url,
                           input_shape = [], 
                           dtype = tf.string,
                           trainable = True)

train_text_list = np.asarray(train_x['text'])
train_text_embedded = hub_layer(train_text_list)
train_text_embedded = train_text_embedded.numpy()

validation_text_list = np.asarray(validation_x['text'])
validation_text_embedded = hub_layer(validation_text_list)
validation_text_embedded = validation_text_embedded.numpy()

train_text_embedded

array([[ 0.5366153 , -1.342566  ,  2.1222613 , ..., -1.0080651 ,
        -0.96196187, -0.45349854],
       [ 2.0242817 , -1.6424292 ,  1.7285795 , ..., -1.4300696 ,
        -2.3009865 , -1.0741191 ],
       [ 0.5674488 ,  0.3319531 ,  0.7694006 , ..., -0.6728438 ,
        -1.0818708 , -0.452739  ],
       ...,
       [-0.4777475 ,  0.8065685 , -0.334889  , ...,  0.6439389 ,
         0.03105509, -0.43927386],
       [-0.29436806,  1.0474839 , -0.69300604, ...,  0.551162  ,
         0.2705982 ,  0.63722724],
       [ 0.8156372 , -2.1133864 ,  0.8432087 , ..., -0.38624156,
        -1.7745864 , -0.46408758]], dtype=float32)

In [155]:
# ARMAMOS EL MODELO
model = tf.keras.Sequential()
model.add(tf.keras.layers.LayerNormalization(input_shape = tuple([train_text_embedded.shape[1]])))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dense(14, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization_29 (Laye (None, 20)                40        
_________________________________________________________________
dense_48 (Dense)             (None, 64)                1344      
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
layer_normalization_30 (Laye (None, 64)                128       
_________________________________________________________________
dense_49 (Dense)             (None, 14)                910       
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 15        
Total params: 2,437
Trainable params: 2,437
Non-trainable params: 0
___________________________________________________

In [156]:
# COMPILAMOS Y ENTRENAMOS

model.save_weights('model.h5') #esto es para luego resetear el entrenamiento

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(train_text_embedded, train_y, batch_size=1024,
                    epochs = 120,
                    verbose = 1)

Epoch 1/120
6/6 [==============================] - 0s 7ms/step - loss: 0.9702 - accuracy: 0.4500
Epoch 2/120
6/6 [==============================] - 0s 5ms/step - loss: 0.7099 - accuracy: 0.5930
Epoch 3/120
6/6 [==============================] - 0s 5ms/step - loss: 0.6146 - accuracy: 0.6577
Epoch 4/120
6/6 [==============================] - 0s 5ms/step - loss: 0.5783 - accuracy: 0.6909
Epoch 5/120
6/6 [==============================] - 0s 7ms/step - loss: 0.5657 - accuracy: 0.7029
Epoch 6/120
6/6 [==============================] - 0s 8ms/step - loss: 0.5566 - accuracy: 0.7204
Epoch 7/120
6/6 [==============================] - 0s 7ms/step - loss: 0.5438 - accuracy: 0.7243
Epoch 8/120
6/6 [==============================] - 0s 5ms/step - loss: 0.5394 - accuracy: 0.7326
Epoch 9/120
6/6 [==============================] - 0s 5ms/step - loss: 0.5395 - accuracy: 0.7288
Epoch 10/120
6/6 [==============================] - 0s 5ms/step - loss: 0.5299 - accuracy: 0.7362
Epoch 11/120
6/6 [===========

In [157]:
# EVALUAMOS CON EL SET DE VALIDACION
loss, precision = model.evaluate(validation_text_embedded, validation_y)
print('Precisión: %.2f' % (precision*100))

72/72 [==============================] - 0s 1ms/step - loss: 0.4921 - accuracy: 0.7763
Precisión: 77.63


In [70]:
# SE REENTRENA CON EL SET COMPLETO

model.load_weights('model.h5') #se resetea el entrenamiento previo

train_complete_text_list = np.asarray(train_df['text'])
train_complete_text_embedded = hub_layer(train_complete_text_list)
train_complete_text_embedded = train_complete_text_embedded.numpy()

history = model.fit(train_complete_text_embedded, target_df, batch_size = 1024,
                   epochs = 100,
                   verbose = 1)

Epoch 1/100
8/8 [==============================] - 0s 6ms/step - loss: 0.7608 - accuracy: 0.5753
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5933 - accuracy: 0.6632
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5522 - accuracy: 0.7238
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5358 - accuracy: 0.7357
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5332 - accuracy: 0.7418
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 0.5249 - accuracy: 0.7458
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5211 - accuracy: 0.7445
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5197 - accuracy: 0.7470
Epoch 9/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5123 - accuracy: 0.7504
Epoch 10/100
8/8 [==============================] - 0s 5ms/step - loss: 0.5111 - accuracy: 0.7549
Epoch 11/100
8/8 [===========

In [29]:
# ARMAMOS LA PREDICCIÓN

text_complete_text_list = np.asarray(test_df['text'])
test_complete_text_embedded = hub_layer(text_complete_text_list)
test_complete_text_embedded = test_complete_text_embedded.numpy()

prediction = (model.predict(test_complete_text_embedded) > 0.5).astype("int32")
prediction

array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [30]:
# EXPORTAMOS EL CSV EN EL FORMATO QUE PIDE KAGGLE
test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>